# Enable GPU acceleration
Go to: 
`Edit > Notebook settings > Hardware accelerator > GPU`

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request
import tarfile

print("Libraries version:")
print(f"Numpy:      {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Tensorflow: {tf.__version__}")

Libraries version:
Numpy:      1.18.5
Matplotlib: 3.2.2
Tensorflow: 2.2.0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
!git config --global user.email '51918753+maximecharriere@users.noreply.github.com'
!git config --global user.name 'maximecharriere'

# Get all files

In [ ]:
%cd /content

## Load Git repo

In [ ]:
if not os.path.isdir("AutonomousRcCar"):
    !git clone https://github.com/maximecharriere/AutonomousRcCar.git

In [ ]:
model_training_dir_path = "/content/AutonomousRcCar/autonomouscar/tf_model_training"

In [ ]:
if not os.path.isdir("models"):
    !git clone https://github.com/tensorflow/models.git

## Load pre-trained model

Choose a tensorflow model:
- [All object detection models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models)
- [Models compatible with Coral Edge TPU](https://coral.ai/models/)

In [ ]:
model_name = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03'

if not os.path.isdir(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    urllib.request.urlretrieve(base_url + model_file, model_file)
    with tarfile.open(model_file) as tar:
        tar.extractall()
    os.remove(model_file)


In [ ]:
fine_tune_checkpoint = os.path.join("/content", model_name, "model.ckpt")

# Install the Tensorflow Object Detection API
It's necessary in the next scripts
https://github.com/tensorflow/models/tree/master/research/object_detection

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
!pip install -q tf_slim

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

# Generate `tfrecord` files

In [ ]:
%cd {model_training_dir_path}

In [ ]:
#Partition data between test and train set
!python3 scripts/partition_dataset.py -i training/images/all -o training/images -r 0.2 -x

In [ ]:
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python3 scripts/xml_to_csv.py -i training/images/train -o training/annotations/train_labels.csv -l training/annotations

# Convert test folder annotation xml files to a single csv.
!python3 scripts/xml_to_csv.py -i training/images/test -o training/annotations/test_labels.csv

In [ ]:
# Generate `train.record`
!python scripts/generate_tfrecord.py --csv_input=training/annotations/train_labels.csv --output_path=training/annotations/train.record --img_path=training/images/train --label_map training/annotations/label_map.pbtxt

# Generate `test.record`
!python scripts/generate_tfrecord.py --csv_input=training/annotations/test_labels.csv --output_path=training/annotations/test.record --img_path=training/images/test --label_map training/annotations/label_map.pbtxt


# Configuring the Training `pipeline.conf`

In [ ]:
def get_num_classes(pbtxt_fname):
    import sys

    sys.path.append("/content/models/research/")
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
test_record_fname = model_training_dir_path + '/training/annotations/test.record'
train_record_fname = model_training_dir_path + '/training/annotations/train.record'
label_map_pbtxt_fname = model_training_dir_path + '/training/annotations/label_map.pbtxt'
batch_size = 24
num_steps = 1000
num_classes = get_num_classes(label_map_pbtxt_fname)
ssd_anchor_generator = '''{
        num_layers: 6
        min_scale: 0.1
        max_scale: 0.6
        aspect_ratios: 1.0
        aspect_ratios: 0.5
        aspect_ratios: 0.3333
      }'''

In [ ]:
%cd /content
base_pipeline_fname = os.path.join(model_name, "pipeline.config")
assert os.path.isfile(base_pipeline_fname), f'`{base_pipeline_fname}` not exist'

In [ ]:
trained_model_path = os.path.join(model_training_dir_path, "training/model")
os.makedirs(trained_model_path, exist_ok=True)

In [ ]:
import re

dest_pipeline_fname = os.path.join(trained_model_path, "pipeline.config")

with open(base_pipeline_fname) as f:
    s = f.read()
with open(dest_pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               f'fine_tune_checkpoint: "{fine_tune_checkpoint}"', s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', f'input_path: "{train_record_fname}"', s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', f'input_path: "{test_record_fname}"', s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', f'label_map_path: "{label_map_pbtxt_fname}"', s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               f'batch_size: {batch_size}', s)
    
    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               f'num_steps: {num_steps}', s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               f'num_classes: {num_classes}', s)

    # Set anchors.
    s = re.sub('ssd_anchor_generator \{.*?\}',
               f'ssd_anchor_generator {ssd_anchor_generator}', s, flags = re.DOTALL)
    f.write(s)

# Start TensorBoard for monitoring

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {trained_model_path}

# Train the model

In [ ]:
!python model_main_tf2.py -- \
  --model_dir={trained_model_path}
  --num_train_steps={num_steps} \
  --sample_1_of_n_eval_examples=1 \
  --pipeline_config_path={dest_pipeline_fname} \
  --alsologtostderr

# Save the model on GitHub and Googe Drive

In [ ]:
backup_path = '/content/gdrive/My Drive/Colab Notebooks/ModelTraining_backup'
!cp -avr {model_training_dir_path+"/training"} "{backup_path}"

In [ ]:
%cd /content/AutonomousRcCar/
!git add --all
!git commit -m 'Update from GoogleColab Notebook'  # commit in Colab
!git push -u origin master          # push to github